# Model Choice with ABCtoolbox
working in `/mnt/ABC_AJmodels_instant_newreal`  
working with files:  
`input_ABCtoolbox_M1_HPC_OSG_2.txt`  
`input_ABCtoolbox_M2_HPC_OSG_2.txt`  
`input_ABCtoolbox_M3_HPC_OSG_2.txt`  
which are a combination of simulations with instantanious growth from HPC and OSG, and have 
1,275,807, 
1,446,125, 
1,343,711 simulations.

In [87]:
import sys, os
os.chdir('/mnt/ABC_AJmodels_instant_newreal')

## Method 1
- filter correlated stats
- run ABCtoolbox method to find best stats for model choice
- use sets of stats with power greater than 0.5

## 1. Find correlated stats
__did not rerun this on jetstream__

Use ABCtoolbox `task estimate` and `pruneCorrelatedStats` on 1,218,497 simulations.  
`ABCtoolbox test_ABC_correlatedstats1.txt`

```
task estimate
simName input_ABCtoolbox_M1_HPC_OSG_2.txt;input_ABCtoolbox_M2_HPC_OSG_2.txt;input_ABCtoolbox_M3_HPC_OSG_2.txt
obsName real_output_M23.summary
params 2-6,8-14,16-22;2-6,8-15,17-24;2-6,8-16,18-26
numRetained 1000
maxReadSims 1218497
outputPrefixe ABC_correlatedstats1_1218497
pruneCorrelatedStats 1
logFile ABC_correlatedstats1_1218497.log
verbose
```

## 2. Prune correlated stats
__did not rerun this on jetstream__

Use python script `subset_stats/main_subset_real.py` and `subset_stats/main_subset_sim.py` to create new file without correlated summary statistics.

```
/vol_c/src/macsswig_simsaj/macss_env/bin/python /vol_c/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_correlatedstats1_100000.log input_ABCtoolbox_M1_HPC_OSG_203.txt remove

/vol_c/src/macsswig_simsaj/macss_env/bin/python /vol_c/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_correlatedstats1_100000.log input_ABCtoolbox_M2_HPC_OSG.txt remove

/vol_c/src/macsswig_simsaj/macss_env/bin/python /vol_c/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_correlatedstats1_100000.log input_ABCtoolbox_M3_HPC_OSG.txt remove
```

__Ran this on jetstream__

```
/mnt/env/simprily_env2.7/bin/python /mnt/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_correlatedstats1_1218497.log real_output_M23_IBD.summary remove
```

Double check the number of stats:  
(One extra value in simulations because of "sim" column)

In [88]:
%%bash
head -1 pruneCorStats_real_output_M23_IBD.summary | tr '\t' '\n' | wc -l
head -1 pruneCorStats_input_ABCtoolbox_M1_HPC_OSG_203.txt | cut -f23- | tr '\t' '\n' | wc -l
head -1 pruneCorStats_input_ABCtoolbox_M2_HPC_OSG.txt | cut -f25- | tr '\t' '\n' | wc -l
head -1 pruneCorStats_input_ABCtoolbox_M3_HPC_OSG.txt | cut -f27- | tr '\t' '\n' | wc -l

53
54
54
54


## 3. Find best stats for model choice with pruned stats
Use ABCtoolbox `task findStatsModelChoice` on 100,000 simulations, 1000 retained simulations, 100 cross validation.  

`ABCtoolbox test_ABC_findStatsModelChoice3.txt`

```
task findStatsModelChoice
simName pruneCorStats_input_ABCtoolbox_M1_HPC_OSG_203.txt;pruneCorStats_input_ABCtoolbox_M2_HPC_OSG.txt;pruneCorStats_input_ABCtoolbox_M3_HPC_OSG.txt
obsName pruneCorStats_real_output_M23.summary
params 2-6,8-14,16-22;2-6,8-15,17-24;2-6,8-16,18-26
numRetained 1000
maxReadSims 100000
modelChoiceValidation 100
diracPeakWidth 0.01
posteriorDensityPoints 100
standardizeStats 0
writeRetained 1
outputPrefix ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_
pruneCorrelatedStats 1
logFile ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100.log
verbose
```

If ABCtoolbox is compled with `openMP`, `findStatsModelChoice` can be run on the number of cores equal to the number of models. The number of statistics is the main component in run time. This is why we prune for highly correlated summary statistics before we find the best statistics for model choice. With 54 summary statistics, 100,000 simulations, 1000 retained simulations, and 100 cross validations it took 1148.7 min to complete.

## 4. Extract best stats for model choice

Use python script `subset_stats/main_subset_real.py` and `subset_stats/main_subset_sim.py` to create new file with top sets of best summary statistics. We kept all sets of summary statistics with power greater than 0.5, which was 108 sets.

This leaves a set of 20 stats.

In [90]:
%%bash
/mnt/env/simprily_env2.7/bin/python /mnt/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt input_ABCtoolbox_M1_HPC_OSG_2.txt keep 108

/mnt/env/simprily_env2.7/bin/python /mnt/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt input_ABCtoolbox_M2_HPC_OSG_2.txt keep 108

/mnt/env/simprily_env2.7/bin/python /mnt/src/macsswig_simsaj/subset_stats/main_subset_sim.py ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt input_ABCtoolbox_M3_HPC_OSG_2.txt keep 108

/mnt/env/simprily_env2.7/bin/python /mnt/src/macsswig_simsaj/subset_stats/main_subset_real.py ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt real_output_M23_IBD.summary keep 108

Reading input file
Extracting simulation id and parameters
Reading summary statistics found in ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt

Creating new file with parameters and summary statistics with high power
Extracting simulation id and parameters
Reading summary statistics found in ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt

Creating new file with parameters and summary statistics with high power
Extracting simulation id and parameters
Reading summary statistics found in ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt

Creating new file with parameters and summary statistics with high power
Extracting simulation id and parameters
Reading summary statistics found in ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt

Creating new file with

In [91]:
!ls keepPowerStats*

keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt
keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt
keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
keepPowerStats_real_output_M23_IBD.summary


In [5]:
%%bash
head -1 keepPowerStats_real_output_M23_IBD.summary | tr '\t' '\n' | sort

Dupl_Af_ASC
FST_AfAs_ASC_m
FST_AfEu_ASC
FST_eAwA_ASC
FST_EuAs_ASC
FST_wAEu_ASC
FST_wAJ_ASC
FST_wAM_ASC
IBD30_num_ME
IBD_median_ME
Pi_WA_ASC
Sing_Af_ASC
Sing_WA_ASC
TajD_Af_ASC
TajD_Af_CGI
TajD_As_ASC
TajD_As_CGI
TajD_Eu_ASC
TajD_M_ASC
TajD_WA_ASC


### 4.b Double check for correlated stats in set of kept best stats for model choice
Use ABCtoolbox `task estimate` and `pruneCorrelatedStats` on 1,275,807 simulations.  
`ABCtoolbox test_ABC_correlatedstats4.b.txt`

```
task estimate
simName keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
obsName keepPowerStats_real_output_M23_IBD.summary
params 2-6,8-14,16-21;2-6,8-15,17-23;2-6,8-16,18-25
numRetained 1000
maxReadSims 1275807
outputPrefix ABC_correlatedstats4b_1275807
pruneCorrelatedStats 1
logFile ABC_correlatedstats4b_1275807.log
verbose
```

Did not find any correlated stats.

## 5. Assess power to choose model, and choose best model with best stats for model choice.
Use ABCtoolbox `task estimate` on 1,275,807 simulations, 1000 retained simulations, 100 cross validation.  
`ABCtoolbox test_ABC_modelchoice5.txt`

```
task estimate
simName keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
obsName keepPowerStats_real_output_M23_IBD.summary
params 2-6,8-14,16-21;2-6,8-15,17-23;2-6,8-16,18-25
numRetained 1000
maxReadSims 1275807
modelChoiceValidation 100
diracPeakWidth 0.01
posteriorDensityPoints 100
standardizeStats 0
writeRetained 0
outputPrefix ABC_modelchoice5_1275807
pruneCorrelatedStats 1
logFile ABC_modelchoice5_1275807.log
verbose
```

Took 15.2833 min to complete.

## Method 2 (sanity check)
- use best stats from original data

### 1. Extract original best stats for model choice

In [9]:
%%bash
ls /mnt/ABC_AJmodels_instant/keepPowerStats_*

/mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_203.txt
/mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt
/mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt
/mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M2_HPC_OSG.txt
/mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
/mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M3_HPC_OSG.txt
/mnt/ABC_AJmodels_instant/keepPowerStats_real_output_M23.summary


In [11]:
%%bash
head -1 /mnt/ABC_AJmodels_instant/keepPowerStats_real_output_M23.summary | tr '\t' '\n' | wc -l

23


In [12]:
%%bash
head -1 /mnt/ABC_AJmodels_instant/keepPowerStats_real_output_M23.summary | tr '\t' '\n' | sort

Dupl_Af_ASC
Dupl_WA_ASC
FST_AfAs_ASC_m
FST_AfEu_ASC
FST_eAwA_ASC
FST_EuAs_ASC
FST_wAEu_ASC
FST_wAJ_ASC
FST_wAM_ASC
IBD30_median_JM
IBD30_median_wAE
IBD30_num_JM
IBD_median_EE
IBD_num_ME
Sing_Af_ASC
Sing_M_ASC
TajD_Af_ASC
TajD_Af_CGI
TajD_As_ASC
TajD_As_CGI
TajD_Eu_ASC
TajD_M_ASC
TajD_WA_ASC


__Compare the original stats and new stats__

In New stats, but not original

In [36]:
%%bash
head -1 /mnt/ABC_AJmodels_instant/keepPowerStats_real_output_M23.summary | tr '\t' '\n' | sort > old
head -1 keepPowerStats_real_output_M23_IBD.summary | tr '\t' '\n' | sort > new
grep -v -f old new
grep -v -f old new | wc -l

IBD30_num_ME
IBD_median_ME
Pi_WA_ASC
Sing_WA_ASC
4


In original stats, but not new

In [31]:
%%bash 
grep -v -f new old
grep -v -f new old | wc -l

Dupl_WA_ASC
IBD30_median_JM
IBD30_median_wAE
IBD30_num_JM
IBD_median_EE
IBD_num_ME
Sing_M_ASC
7


Shared in original and new

In [39]:
%%bash
grep -f old new
grep -f old new | wc -l

rm old
rm new

Dupl_Af_ASC
FST_AfAs_ASC_m
FST_AfEu_ASC
FST_eAwA_ASC
FST_EuAs_ASC
FST_wAEu_ASC
FST_wAJ_ASC
FST_wAM_ASC
Sing_Af_ASC
TajD_Af_ASC
TajD_Af_CGI
TajD_As_ASC
TajD_As_CGI
TajD_Eu_ASC
TajD_M_ASC
TajD_WA_ASC
16


In [59]:
os.chdir('/mnt/ABC_AJmodels_instant_newreal/ModelChoice_orig_stats')

Use python script subset_stats/main_subset_real.py to create new file with top sets of best summary statistics from original data. We kept all sets of summary statistics with power greater than 0.5 from the original data in `/mnt/ABC_AJmodels_instant`, which was 77 sets.

This leaves a set of 23 stats.

In [62]:
%%bash
pwd
cp /mnt/ABC_AJmodels_instant/keepPowerStats_input_ABCtoolbox_M*2.txt .
cp ../real_output_M23_IBD.summary .

/mnt/ABC_AJmodels_instant_newreal/ModelChoice_orig_stats


In [63]:
!ls -ltr

total 2115200
-rw-r--r-- 1 aglad aglad 632393999 Apr 21 16:27 keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt
-rw-r--r-- 1 aglad aglad 819970964 Apr 21 16:27 keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt
-rw-r--r-- 1 aglad aglad 713570554 Apr 21 16:27 keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
-rw-rw-r-- 1 aglad aglad      4126 Apr 21 16:27 real_output_M23_IBD.summary


In [64]:
%%bash
/mnt/env/simprily_env2.7/bin/python /mnt/src/macsswig_simsaj/subset_stats/main_subset_real.py /mnt/ABC_AJmodels_instant/ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt real_output_M23_IBD.summary keep 77

Reading input file
Reading summary statistics found in /mnt/ABC_AJmodels_instant/ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_greedySearchForBestStatisticsForModelChoice.txt

Creating new file with summary statistics with high power


In [65]:
!ls -ltr

total 2115204
-rw-r--r-- 1 aglad aglad 632393999 Apr 21 16:27 keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt
-rw-r--r-- 1 aglad aglad 819970964 Apr 21 16:27 keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt
-rw-r--r-- 1 aglad aglad 713570554 Apr 21 16:27 keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
-rw-rw-r-- 1 aglad aglad      4126 Apr 21 16:27 real_output_M23_IBD.summary
-rw-rw-r-- 1 aglad aglad       548 Apr 21 16:27 keepPowerStats_real_output_M23_IBD.summary


In [66]:
%%bash
head -1 keepPowerStats_real_output_M23_IBD.summary | tr '\t' '\n' | sort
head -1 keepPowerStats_real_output_M23_IBD.summary | tr '\t' '\n' | wc -l

Dupl_Af_ASC
Dupl_WA_ASC
FST_AfAs_ASC_m
FST_AfEu_ASC
FST_eAwA_ASC
FST_EuAs_ASC
FST_wAEu_ASC
FST_wAJ_ASC
FST_wAM_ASC
IBD30_median_JM
IBD30_median_wAE
IBD30_num_JM
IBD_median_EE
IBD_num_ME
Sing_Af_ASC
Sing_M_ASC
TajD_Af_ASC
TajD_Af_CGI
TajD_As_ASC
TajD_As_CGI
TajD_Eu_ASC
TajD_M_ASC
TajD_WA_ASC
23


## 2. Assess power to choose model, and choose best model with original best stats for model choice.
Use ABCtoolbox `task estimate` on 1,275,807 simulations, 1000 retained simulations, 100 cross validation.  

Took 15.2833 min to complete.

In [71]:
%%bash
echo "task estimate
simName keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
obsName keepPowerStats_real_output_M23_IBD.summary
params 2-6,8-14,16-21;2-6,8-15,17-23;2-6,8-16,18-25
numRetained 1000
maxReadSims 1275807
modelChoiceValidation 100
diracPeakWidth 0.01
posteriorDensityPoints 100
standardizeStats 0
writeRetained 0
outputPrefix ABC_modelchoice5_1275807
pruneCorrelatedStats 1
logFile ABC_modelchoice5_1275807.log
verbose" >test_ABC_modelchoice.txt

In [70]:
%%bash
ABCtoolbox test_ABC_modelchoice.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_modelchoice.txt' ... done!
- Writing log to 'ABC_modelchoice5_1275807.log'
- Initializing random generator ... done with seed 158366981!
- Reading observed data file 'keepPowerStats_real_output_M23_IBD.summary' ... done!
   -> 1 data sets with 23 statistics each.
- Reading files with simulations:
   - Reading simulations of set 1:
      - Reading up to 1275807 simulations from file 'keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt' ... done!   
         -> 1275806 simulations with 18 parameters and 23 statistics each.
      - Pruning highly correlated statsitics:
         - Calculate pairwise correlations ...done!
            -> 0 statistics have been removed!
   - Reading simulations of set 2:
      - Reading up to 1275807 simulations from file 'keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt' ... done!   
         -> 1275807 simulations with 20 parameters and 23 statistics each.
      - Pruning highly correlated statsi

## Method 3
- filter correlated stats
- run ABCtoolbox method to find best stats for model choice
- use sets of stats with power greater than 0.5
- reduce parameters to infer

In [93]:
os.chdir('/mnt/ABC_AJmodels_instant_newreal')
!mkdir -p ReduceParams
os.chdir('/mnt/ABC_AJmodels_instant_newreal/ReduceParams')
!pwd

/mnt/ABC_AJmodels_instant_newreal/ReduceParams


In [94]:
%%bash
cp ../keep* .
ls

keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt
keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt
keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
keepPowerStats_real_output_M23_IBD.summary
M1_params
M2_params
M3_params
params_not_use.txt
test_ABC_modelchoice.txt


### 1.  Find parameter indexes

__Do not use parameters:__
- Asc_NAF
- Asc_NEU
- Asc_NCHB
- daf
- Log10_NAF
- Log10_NANC
- Log10_NCEU
- Log10_NCHB
- Tgrowth_Af
- TAF
- Teu_as
- TAg

In [81]:
%%bash
echo "Asc_NAF
Asc_NEU
Asc_NCHB
daf
Log10_NAF
Log10_NANC
Log10_NCEU
Log10_NCHB
Tgrowth_Af
TAF
Teu_as
TAg" >params_not_use.txt
ls

M1_params
M2_params
M3_params
params_not_use.txt


In [82]:
%%bash
head -1 ../input_ABCtoolbox_M1_HPC_OSG_2.txt | cut -f1-22 | tr '\t' '\n' >M1_params
head -1 ../input_ABCtoolbox_M2_HPC_OSG_2.txt | cut -f1-24 | tr '\t' '\n' >M2_params
head -1 ../input_ABCtoolbox_M3_HPC_OSG_2.txt | cut -f1-26 | tr '\t' '\n' >M3_params

In [83]:
%%bash
grep -n -v -f params_not_use.txt M1_params

1:sim
10:Log10_NA
11:Log10_NAg
12:Log10_NJ
13:Log10_NM
14:m
17:TEM
19:TA
20:TMJ
21:Tm


10-14,17,19-21

In [84]:
%%bash
grep -n -v -f params_not_use.txt M2_params

1:sim
10:Log10_NWA
11:Log10_NEA
12:Log10_NAg
13:Log10_NJ
14:Log10_NM
15:m
18:TEM
20:TA
21:TMJ
22:TAEW
23:Tm


10-15,18,20-23

In [85]:
%%bash
grep -n -v -f params_not_use.txt M3_params

1:sim
10:Log10_NWA
11:Log10_NEA
12:Log10_NAg
13:Log10_NJ
14:Log10_NM
15:mE
16:mW
19:TEM
21:TA
22:TMJ
23:TAEW
24:TmE
25:TmW


10-16,19,21-25

## 2. Find best stats for model choice with pruned stats
Use ABCtoolbox `task findStatsModelChoice` on 100,000 simulations, 1000 retained simulations, 100 cross validation.  

`ABCtoolbox test_ABC_findStatsModelChoice3.txt`

```
task findStatsModelChoice
simName ../pruneCorStats_input_ABCtoolbox_M1_HPC_OSG_203.txt;../pruneCorStats_input_ABCtoolbox_M2_HPC_OSG.txt;../pruneCorStats_input_ABCtoolbox_M3_HPC_OSG.txt
obsName ../pruneCorStats_real_output_M23_IBD.summary
params 10-14,17,19-21;10-15,18,20-23;10-16,19,21-25
numRetained 1000
maxReadSims 100000
modelChoiceValidation 100
diracPeakWidth 0.01
posteriorDensityPoints 100
standardizeStats 0
writeRetained 1
outputPrefix ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100_
pruneCorrelatedStats 1
logFile ABC_searchStatsForModelChoice_pruneCorStats_100000_1000_100.log
verbose
```

If ABCtoolbox is compled with `openMP`, `findStatsModelChoice` can be run on the number of cores equal to the number of models. The number of statistics is the main component in run time. This is why we prune for highly correlated summary statistics before we find the best statistics for model choice. With 54 summary statistics, 100,000 simulations, 1000 retained simulations, and 100 cross validations it took 1148.7 min to complete.

## 3. Assess power to choose model, and choose best model with best stats for model choice.
Use ABCtoolbox `task estimate` on 1,275,807 simulations, 1000 retained simulations, 100 cross validation.  
`ABCtoolbox test_ABC_modelchoice.txt`

In [97]:
%%bash
echo "task estimate
simName keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt;keepPowerStats_input_ABCtoolbox_M3_HPC_OSG_2.txt
obsName keepPowerStats_real_output_M23_IBD.summary
params 10-14,17,19-21;10-15,18,20-23;10-16,19,21-25
numRetained 1000
maxReadSims 1275807
modelChoiceValidation 1000
diracPeakWidth 0.01
posteriorDensityPoints 100
standardizeStats 0
writeRetained 0
outputPrefix ABC_modelchoice_1275807
pruneCorrelatedStats 1
logFile ABC_modelchoice_1275807.log
verbose" >test_ABC_modelchoice.txt

In [99]:
%%bash
ABCtoolbox test_ABC_modelchoice.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_modelchoice.txt' ... done!
- Writing log to 'ABC_modelchoice_1275807.log'
- Initializing random generator ... done with seed 11068684!
- Reading observed data file 'keepPowerStats_real_output_M23_IBD.summary' ... done!
   -> 1 data sets with 20 statistics each.
- Reading files with simulations:
   - Reading simulations of set 1:
      - Reading up to 1275807 simulations from file 'keepPowerStats_input_ABCtoolbox_M1_HPC_OSG_2.txt' ... done!   
         -> 1275806 simulations with 9 parameters and 20 statistics each.
      - Pruning highly correlated statsitics:
         - Calculate pairwise correlations ...done!
            -> 0 statistics have been removed!
   - Reading simulations of set 2:
      - Reading up to 1275807 simulations from file 'keepPowerStats_input_ABCtoolbox_M2_HPC_OSG_2.txt' ... done!   
         -> 1275807 simulations with 11 parameters and 20 statistics each.
      - Pruning highly correlated statsitic

## Method 4
- filter correlated stats
- Manually pick stats theoretically informative for model choice